In [1]:
pip install xgboost tensorflow pandas pyswarm

  Using cached tensorflow-2.17.0-cp312-cp312-win_amd64.whl.metadata (3.2 kB)
  Using cached tensorflow_intel-2.17.0-cp312-cp312-win_amd64.whl.metadata (5.0 kB)
Using cached tensorflow-2.17.0-cp312-cp312-win_amd64.whl (2.0 kB)
Using cached tensorflow_intel-2.17.0-cp312-cp312-win_amd64.whl (385.2 MB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from pyswarm import pso
import xgboost as xgb
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load the dataset
data = pd.read_csv(r"C:\Users\prave\OneDrive\Desktop\diabetis\diabetes_new_update.csv")
print(data.head())

# Features and target variable
X = data.drop('Outcome', axis=1)
y = data['Outcome']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for PSO
def objective_function(hyperparams):
    learning_rate, max_depth = hyperparams
    
    print(f"Evaluating: learning_rate={learning_rate}, max_depth={max_depth}")
    
    # Train XGBoost model with the hyperparameters
    xgb_model = xgb.XGBClassifier(learning_rate=learning_rate, max_depth=int(max_depth))
    
    try:
        xgb_model.fit(X_train, y_train)
        preds = xgb_model.predict(X_test)
        accuracy = accuracy_score(y_test, preds)
        print(f"Accuracy: {accuracy}")
        return -accuracy  # Minimize negative accuracy to maximize accuracy
    except Exception as e:
        print(f"Error: {e}")
        return 1e10  # Return a large value in case of error

# Define bounds for the hyperparameters (learning_rate, max_depth)
lb = [0.01, 2]  # Lower bounds
ub = [0.3, 10]  # Upper bounds

# Run PSO to optimize the XGBoost hyperparameters
best_hyperparams, best_score = pso(objective_function, lb, ub)

# Output the best hyperparameters
print("Best Hyperparameters: ", best_hyperparams)
print("Best Score: ", -best_score)

# Train XGBoost with the best hyperparameters
xgb_model = xgb.XGBClassifier(learning_rate=best_hyperparams[0], max_depth=int(best_hyperparams[1]))
xgb_model.fit(X_train, y_train)
xgb_preds = xgb_model.predict(X_test)

# Define the DNN model
dnn_model = Sequential()
dnn_model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
dnn_model.add(Dense(32, activation='relu'))
dnn_model.add(Dense(1, activation='sigmoid'))

# Compile the DNN model
dnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the DNN model
dnn_model.fit(X_train, y_train, epochs=50, batch_size=10, validation_data=(X_test, y_test))

# Get predictions from the DNN model
dnn_preds = dnn_model.predict(X_test).flatten()

# Combine predictions (using weighted average for example)
final_preds = (0.5 * dnn_preds) + (0.5 * xgb_preds)

# Thresholding the final predictions
final_preds = [1 if pred > 0.5 else 0 for pred in final_preds]

# Evaluate accuracy of the hybrid model
print("Final Hybrid Model Accuracy:", accuracy_score(y_test, final_preds))


   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            2      138             62             35        0  33.6   
1            0       84             82             31      125  38.2   
2            0      145              0              0        0  44.2   
3            0      135             68             42      250  42.3   
4            1      139             62             41      480  40.7   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.127   47        1  
1                     0.233   23        0  
2                     0.630   31        1  
3                     0.365   24        1  
4                     0.536   21        0  
Evaluating: learning_rate=0.11956765752587224, max_depth=7.741263738684164
Accuracy: 0.98
Evaluating: learning_rate=0.2489060745584761, max_depth=6.017705479498304
Accuracy: 0.985
Evaluating: learning_rate=0.08019247094348896, max_depth=6.678351302822507
Accuracy: 0.965
Evaluating: learning_rate=0.2840